In [ ]:
import pandas as pd

path = "/workspace/zhangzitian/code/verl/data/r1_bench/generation_results/qwen2.5_7b-math56_code38-r1_bench-test-output-8.parquet"

df = pd.read_parquet(path)
# print(df.head())

TypeError: string indices must be integers

In [7]:
df.iloc[0]["prompt"]

array([{'content': 'Let $x,y$ and $z$ be positive real numbers that satisfy the following system of equations: \n\\[\\log_2\\left({x \\over yz}\\right) = {1 \\over 2}\\]\n\\[\\log_2\\left({y \\over xz}\\right) = {1 \\over 3}\\]\n\\[\\log_2\\left({z \\over xy}\\right) = {1 \\over 4}\\]\nThen the value of $\\left|\\log_2(x^4y^3z^2)\\right|$ is $\\tfrac{m}{n}$ where $m$ and $n$ are relatively prime positive integers. Find $m+n$.', 'role': 'user'}],
      dtype=object)